In [38]:
%%capture
!pip install git+https://github.com/openai/swarm.git
%pip install gradio_client

In [ ]:
import openai, os
os.environ["OPENAI_API_KEY"] = ""
hf_token = ""

In [46]:
from swarm import Agent
import pandas as pd
import openpyxl

# Load data from various sources
def load_acs_data():
    df = pd.read_csv("SampleACS.csv")
    df['Customer ID'] = df['Customer ID'].astype(str)
    return df

def load_olt_data_dictionary():
    return pd.read_excel("OLT Data Dictionary.xlsx")

def load_crm_data():
    df = pd.read_csv("CRM dataset.csv")
    df['Customer ID'] = df['Customer ID'].astype(str)
    return df

def load_olt_status_and_alarms():
    df = pd.read_excel("OLT status and alarms.xlsx")
    # df['CustomerID'] = df['CustomerID'].astype(str)
    return df

def load_ticketing_data():
    df = pd.read_excel("Ticketing sample data.xlsx")
    df['CustomerAccount'] = df['CustomerAccount'].astype(str)
    return df

def knowledgegraph_tool(query):
  from gradio_client import Client
  client = Client("neuronslabs/Telcom_GraphRAG_Agent", hf_token)
  result = client.predict(
      history=[[query, None]],
      api_name="/bot"
  )
  return(result[-1][-1])

# Integrated customer data
class IntegratedCustomerData:
    def __init__(self):
        self.acs_data = load_acs_data()
        self.crm_data = load_crm_data()
        self.olt_status = load_olt_status_and_alarms()
        self.ticketing_data = load_ticketing_data()

    def get_customer_info(self, customer_id):
        customer_info = {}

        if customer_id in self.crm_data['Customer ID'].values:
            customer_info['CRM'] = self.crm_data[self.crm_data['Customer ID'] == customer_id].to_dict('records')[0]

        if customer_id in self.acs_data['Customer ID'].values:
            customer_info['ACS'] = self.acs_data[self.acs_data['Customer ID'] == customer_id].to_dict('records')[0]

        if customer_id in self.olt_status['olt_id'].values:
            customer_info['OLT_Status'] = self.olt_status[self.olt_status['olt_id'] == customer_id].to_dict('records')[0]

        if customer_id in self.ticketing_data['CustomerAccount'].values:
            customer_info['Tickets'] = self.ticketing_data[self.ticketing_data['CustomerAccount'] == customer_id].to_dict('records')

        return customer_info

integrated_data = IntegratedCustomerData()

# Agent functions
def check_olt_status(customer_id):
    """Check the status of OLT for a specific customer."""
    customer_info = integrated_data.get_customer_info(customer_id)
    if 'OLT_Status' in customer_info:
        return f"The OLT status for customer {customer_id} is: {customer_info['OLT_Status']['Status']}"
    else:
        return f"No OLT status information found for customer {customer_id}"

def get_customer_info(customer_id):
    """Retrieve integrated customer information."""
    customer_info = integrated_data.get_customer_info(customer_id)
    if customer_info:
        return f"Integrated Customer Info for {customer_id}: {customer_info}"
    else:
        return f"No information found for customer {customer_id}"

def create_ticket(customer_id, issue):
    """Create a new support ticket."""
    new_ticket_id = len(integrated_data.ticketing_data) + 1
    print(f"[mock] Creating ticket {new_ticket_id} for customer {customer_id}: {issue}")
    return f"Ticket {new_ticket_id} created successfully for customer {customer_id}."

def get_acs_info(customer_id):
    """Retrieve ACS-specific information for a customer."""
    customer_info = integrated_data.get_customer_info(customer_id)
    if 'ACS' in customer_info:
        return f"ACS Info for customer {customer_id}: {customer_info['ACS']}"
    else:
        return f"No ACS information found for customer {customer_id}"

# Define agents
olt_agent = Agent(
    name="OLT Agent",
    instructions="Handle OLT-related queries and status checks. Summarise the output in better readble format.",
    functions=[check_olt_status, load_olt_data_dictionary]
)

customer_agent = Agent(
    name="CRM Agent",
    instructions="Handle queries related to the Customer from CRM data. Summarise the output in better readble format.",
    functions=[get_customer_info]
)

acs_agent = Agent(
    name="ACS Agent",
    instructions="Handle queries related to the Auto Configuration Server (ACS) data. Summarise the output in better readble format.",
    functions=[get_acs_info]
)

solution_agent = Agent(
    name="Solution Agent",
    instructions="Handle queries related to solving the customer problems, like fixing router based on customer data. Summarise the output in better readble format.",
    functions=[knowledgegraph_tool]
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
)

# Transfer functions
def transfer_to_olt():
    return olt_agent

def transfer_to_customer():
    return customer_agent

def transfer_to_acs():
    return acs_agent

def transfer_to_solution():
    return solution_agent


def transfer_back_to_triage():
    return triage_agent

# Set up transfer functions for each agent
triage_agent.functions = [transfer_to_olt, transfer_to_customer, transfer_to_acs, transfer_to_solution]
olt_agent.functions.append(transfer_back_to_triage)
customer_agent.functions.append(transfer_back_to_triage)
acs_agent.functions.append(transfer_back_to_triage)
solution_agent.functions.append(transfer_back_to_triage)

In [51]:
from swarm import Swarm
client = Swarm()

In [57]:
query = "give me data on customer 100005"
response = client.run(
   agent=triage_agent,
   messages=[{"role":"user", "content": query }]
)
print(response.messages[-1]["content"])

Here is the detailed customer information for Customer ID: 100005:

### Personal and Account Details:
- **Name:** David Wong
- **Email:** davidw@email.com
- **Phone Number:** +65 8765 4321
- **Mailing Address:** Block 18, Oak Road, Singapore 200001
- **Service Address:** Block 42, Oak Road, Singapore 120001
- **Account Number:** 20005
- **Account Status:** Active
- **Service Plan:** Fibre 500Mbps
- **Installation Date:** 2023-08-05, 09:47
- **Activation Date:** 2023-08-05, 11:15

### Billing Information:
- **Billing Cycle:** Monthly
- **Billing Address:** Block 95, Main Street, Singapore 220001
- **Payment Method:** Bank Account
- **Payment Due Date:** 10th of each month
- **Payment History:** On time payments, no late fees
- **Outstanding Balance:** 0

### Technical Details (Modem/Router):
- **Model:** Netgear Nighthawk RAX120
- **Serial Number:** X3GASCYN5R
- **Installation Equipment:** WiFi Router
- **Additional Services:** Internet

### Customer Scores:
- **Churn Risk Score:** 3
- 

In [48]:
from swarm.repl import run_demo_loop

if __name__ == "__main__":
    run_demo_loop(triage_agent)

Starting Swarm CLI 🐝
User: cutomer data from crm for 100005
Triage Agent: transfer_to_customer()
CRM Agent: get_customer_info("customer_id"= "100005")
CRM Agent: Here's a summary of the customer data retrieved from CRM for Customer ID 100005:

### Personal Information
- **Name:** David Wong
- **Email:** davidw@email.com
- **Phone:** +65 8765 4321
- **Mailing Address:** Block 18, Oak Road, Singapore 200001

### Account Details
- **Account Number:** 20005
- **Status:** Active
- **Type:** Business
- **Service Plan:** Fibre 500Mbps
- **Installation Date:** 2023-08-05
- **Activation Date:** 2023-08-05
- **Billing Cycle:** Monthly
- **Billing Address:** Block 95, Main Street, Singapore 220001
- **Payment Method:** Bank Account
- **Payment Due Date:** 10th of each month
- **Payment History:** On-time payments, no late fees
- **Outstanding Balance:** $0

### Equipment Information
- **Modem/Router:** Netgear Nighthawk RAX120
- **Serial Number:** X3GASCYN5R

### Service Information
- **Additiona

KeyboardInterrupt: Interrupted by user